v0.1, json output


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [ ]:
chrome_options = Options()
#stops the browser from popping up when this program runs 
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path="chromedriver", chrome_options=chrome_options)

In [ ]:
CATALOGUE_LINK_1ST_HALF = 'https://open.bccampus.ca/browse-our-collection/find-open-textbooks/?start=' 
list_number = 0 
CATALOGUE_LINK_2ND_HALF = '&subject=&contributor=&searchTerm=&keyword=' 

catalogue_all_links = [] 
#prepare links of catalogue to iterate through
for i in range(0, 390, 10): 
    catalogue_all_links.append(CATALOGUE_LINK_1ST_HALF + str(i) + CATALOGUE_LINK_2ND_HALF)

In [ ]:
list_of_links_of_all_books = [] 
BOOK_LINK_1ST_HALF = 'https://open.bccampus.ca/browse-our-collection/find-open-textbooks/'
#iterate through all 38 pages of catalogues
for link in catalogue_all_links:
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    #get links of each books in the page
    for link in soup.findAll('a', href=True):
        if ('uuid' in link.attrs.get('href')): 
            list_of_links_of_all_books.append(BOOK_LINK_1ST_HALF + link.attrs.get('href'))

#remove duplicates 
list_of_links_of_all_books = list(dict.fromkeys(list_of_links_of_all_books))

In [ ]:
#initialise arrays to store values of each book. prepare for insertion into json. 
names = [] 
genres = [] 
publish_dates = [] 
descriptions = [] 
authors = [] 
photosLinks = [] 
links = [] 
Ids = []

In [ ]:
def formaturl(url):
    if not re.match('(?:http|ftp|https):', url):
        return 'http:{}'.format(url)
    return url

def setGenre(link): 
    if ('Biological' or 'Astronomy' or 'Biology' or 'Chemistry' or 'Physics' or 'Science') in link: 
        return 'Biological/Physical Sciences'
    elif ('Business' or 'Career' or 'College' or 'Accounting' or 'Business' or 'Commerce' or 'Entrepreneurship' or 'Finance' or 'Human' or 'Management' or 'Non-Profits') in link: 
        return 'Business'
    elif ('Computer' or 'Algorithms' or 'Computer' or 'Databases' or 'Systems' or 'Web') in link: 
        return 'Computer Sciences' 
    elif ('Humanities' or 'English' or 'Gender' or 'History' or 'Lingustics' or 'Philosophy' or 'Spanish') in link:
        return 'Humanities'
    elif ('Math' or 'Algebra' or 'Calculus' or 'Math' or 'Statistics') in link: 
        return 'Math/Stats'
    else: 
        return 'Others'

#iterate through links to each book to get information of books and store them in the arrays
number = 1 
for link in list_of_links_of_all_books: 
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    #parse a webpage for book details 
    bookDetailsHTML = soup.find(attrs={'itemtype':'https://schema.org/Book'})

    #get required values from the webpage 
    name = bookDetailsHTML.find(attrs={'itemprop':'name'}).text
    names.append(name)

    #substring to keep date only
    publish_date = bookDetailsHTML.find(attrs={'itemprop':'datePublished'}).attrs.get('content')[:10]
    publish_dates.append(publish_date)

    description = bookDetailsHTML.find(attrs={'itemprop':'description'}).text
    descriptions.append(description)

    author = bookDetailsHTML.find(attrs={'itemprop':'author copyrightHolder'}).text
    authors.append(author)

    #link to image. (logic at backend should check if 404, give no image)
    photo = bookDetailsHTML.find(attrs={'itemprop':'image'}).attrs.get('src')
    photosLinks.append(photo)
    
    #Id are manually assigned
    Ids.append(number)
    #download image file. photo string is image filename. 
    image_url = soup.find(attrs={'itemprop':'image'}).attrs.get('src')
    image_url = formaturl(image_url)
    #print(image_url)
    img_data = requests.get(image_url).content
    
    with open('book_' + str(number) + '.jpg', 'wb') as handler:
        handler.write(img_data)
    links.append('book_' + str(number) + '.jpg')
    #links.append(link) #in case switch to download on demand (user access)

    #get genre from subject areas 
    print('subject area text:' + soup.find(attrs={'itemprop':"about"}).text)
    print('genre appended:' + setGenre(soup.find(attrs={'itemprop':"about"}).text))
    genres.append(setGenre(soup.find(attrs={'itemprop':"about"}).text))

    print('link ' + str(number) + ' done') 
    number = number + 1 

In [ ]:
print(len(list_of_links_of_all_books))
print(len(names))
print(len(publish_dates))
print(len(descriptions))
print(len(authors))
print(len(photosLinks))
print(len(links))

In [ ]:
import json

data = [{'Id': id, 'title': name, 'description': description, 'photo': photosLink, 'author': author, 'link':link, 'publishDate':publish_date, 'genre': genre} 
    for id, name, description, photosLink, author, link, publish_date, genre
    in zip(Ids, names, descriptions, photosLinks, authors, links, publish_dates, genres)]


with open('scrapedTextbooks.json', 'w') as outfile:
    json.dump(data,
        outfile, 
        indent=4)

In [ ]:
list_of_links_of_all_books

In [ ]:
link = 'https://open.bccampus.ca/browse-our-collection/find-open-textbooks/?uuid=e1c00bd0-99f2-4842-aa80-2abccd5cee29&contributor=&keyword=&subject='
driver.get(link)
content = driver.page_source
soup = BeautifulSoup(content)
soup.find(attrs={'itemprop':"about"}).text

In [ ]:
genres